In [26]:
# Resources
import sqlite3
import pandas as pd

# Create a SQL connection to SQLite database
con = sqlite3.connect("../../data/LIFE_EXPECTANCY_DB.db")
cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM LIFE_EXPECTANCY_TABLE LIMIT 10;'):
    print(row)

# Read sqlite query results into a pandas DataFrame
life_df = pd.read_sql_query("SELECT * from LIFE_EXPECTANCY_TABLE", con)

# Be sure to close the SQLite db connection
con.close()

# Verify that result of SQL query is stored in the dataframe
print(life_df.head(5))

# Dataframe types
print(life_df.dtypes)

# Dataframe types
print(life_df.dtypes)

('Afghanistan', 2015, 'Developing', 65.0, 263, 62, 0.01, 71.27962362, 65, 1154, 19.1, 83, 6, 8.16, 65, 0.1, 584.25921, 33736494, 17.2, 17.3, 0.479, 10.1)
('Afghanistan', 2014, 'Developing', 59.9, 271, 64, 0.01, 73.52358168, 62, 492, 18.6, 86, 58, 8.18, 62, 0.1, 612.696514, 327582, 17.5, 17.5, 0.476, 10.0)
('Afghanistan', 2013, 'Developing', 59.9, 268, 66, 0.01, 73.21924272, 64, 430, 18.1, 89, 62, 8.13, 64, 0.1, 631.744976, 31731688, 17.7, 17.7, 0.47, 9.9)
('Afghanistan', 2012, 'Developing', 59.5, 272, 69, 0.01, 78.1842153, 67, 2787, 17.6, 93, 67, 8.52, 67, 0.1, 669.959, 3696958, 17.9, 18.0, 0.463, 9.8)
('Afghanistan', 2011, 'Developing', 59.2, 275, 71, 0.01, 7.097108703, 68, 3013, 17.2, 97, 68, 7.87, 68, 0.1, 63.537231, 2978599, 18.2, 18.2, 0.454, 9.5)
('Afghanistan', 2010, 'Developing', 58.8, 279, 74, 0.01, 79.67936736, 66, 1989, 16.7, 102, 66, 9.2, 66, 0.1, 553.32894, 2883167, 18.4, 18.4, 0.448, 9.2)
('Afghanistan', 2009, 'Developing', 58.6, 281, 77, 0.01, 56.76221682, 63, 2861, 16.2